# Project Management

### Param

In [52]:
_pathTasks='tasks.csv'
_pathAvail='avail.csv'

## Internals

In [3]:
import pandas as pd
import datetime as dt

### Load data

In [82]:
_TaskHeaders = ['TaskId','Name','WorkLoad','Who','StartDate','EndDate','Priority']

def LoadTask(path):
        data = pd.read_csv(path, header=0,names=_TaskHeaders)
        data.index=data.TaskId
        return data
def LoadAvailability(path):
    '''
    This function load an availability matrix 
    header : who, dates+
    It expects first column to be who, the other columns availabity per day in terme of days
    '''
    availCSV = pd.read_csv(path)
    datesHeaders = availCSV.columns[1:]    
    cols = ['Who']
    cols.append([dt.datetime.strptime(date, '%d/%m/%Y').date() for date in datesHeaders])
    availCSV.columns =cols
    return availCSV
    

### Save

In [ ]:
def SaveData(data,path):
    data.to_csv(path)

### Domain
* Find tasks of a personn
* Order tasks by priority then ID (just in case)
* Compute end dates
    * Find previous task of personn
    * Based on a start date and workload, compute end date

In [122]:
def FindPreviousTaskOfSomeone(tasks, current):
    '''
    Find the previous task
    Expects an order tasks pandas by priority desc and taskid 
    '''
    whosTasks = FindTasksOfSomeone(tasks,current.Who)    
    prevTasks= whosTasks.loc[:current.TaskId]
    if len(prevTasks)<2 :
        return None
    else :
        return prevTasks.iloc[-2]
    
    

In [61]:
def FindTasksOfSomeone(tasks, who):
    return tasks[tasks.Who==who]
def FindTaskIdById(tasks, taskid):
    return tasks[tasks.TaskId=taskid].head()
def OrderTaskByPriority(tasks):
    return tasks.sort_values(by=['Priority','TaskId'], ascending=[False,True])

## Load Data

In [97]:
_tasks = LoadTask(_pathTasks)

In [54]:
_avail = LoadAvailability(_pathAvail)

In [98]:
_tasks.head(10)

,TaskId,Name,WorkLoad,Who,StartDate,EndDate,Priority
TaskId,,,,,,,
1,1,Test 1,4,CGA,NaN,NaN,150
2,2,Test 2,4,CGA,NaN,NaN,130
3,3,Test 3,4,ACH,NaN,NaN,140


In [56]:
_avail.head(1)

,Who,"(2016-05-01,)"
0,'CGA',1


In [99]:
_tasks = OrderTaskByPriority(_tasks)
_tasks.head(10)

,TaskId,Name,WorkLoad,Who,StartDate,EndDate,Priority
TaskId,,,,,,,
1,1,Test 1,4,CGA,NaN,NaN,150
3,3,Test 3,4,ACH,NaN,NaN,140
2,2,Test 2,4,CGA,NaN,NaN,130


In [100]:
whosTasks = FindTasksOfSomeone(_tasks,'CGA')    
whosTasks.head(10)

,TaskId,Name,WorkLoad,Who,StartDate,EndDate,Priority
TaskId,,,,,,,
1,1,Test 1,4,CGA,NaN,NaN,150
2,2,Test 2,4,CGA,NaN,NaN,130


In [109]:
_tasks.loc[:3].tail(1)

,TaskId,Name,WorkLoad,Who,StartDate,EndDate,Priority
TaskId,,,,,,,
3,3,Test 3,4,ACH,NaN,NaN,140


In [124]:
_tasks.iloc[1]

TaskId            3
Name         Test 3
WorkLoad          4
Who             ACH
StartDate       NaN
EndDate         NaN
Priority        140
Name: 3, dtype: object

In [126]:
FindPreviousTaskOfSomeone(_tasks,_tasks.iloc[1])